## [5주차_기다연] 순환신경망 RNN
1. 데이터셋 수집
2. 태깅, 형태소 분석, 데이터 전처리
3. word2index, index2word 정의
3. 문장 인코딩
4. 원핫 인코딩
5. LSTM 이용한 챗봇 모델
6. 예측 답변 생성

#### **과제 내용**
5주차 순환신경망(RNN) 단원의 과제는 아래 사이트들을 참고하여 Vanilla RNN 모델로 1개의 NLP task를 구현해보는 것이다.
<br>
참고 사이트 :
1. [wikidocs] https://wikidocs.net/22894
 - 스팸 메일 분류하기: 이진 분류(Binary Classification)

2.  [wikidocs] https://wikidocs.net/24586, https://wikidocs.net/44249, https://wikidocs.net/94600
 - 리뷰 감성 분류하기: 감성 분류(Sentiment Analysis)

3. [Kaggle] https://www.kaggle.com/roblexnana/generating-text-for-nlp-using-simplernn-with/
 - 텍스트 생성하기: 언어 모델(Language Models), 텍스트 입력시 그 다음에 나올 내용을 생성해주는 언어 모델
 - 문자(character) 단위/단어 단위/문장 단위도 가능

4. [Kaggle] https://www.kaggle.com/ozkanozturk/stock-price-prediction-by-simple-rnn-and-lstm
 - 주가 예측하기: 시계열 데이터 분석 및 예측 (Tesla)

5. [wikidocs] https://wikidocs.net/77246, https://diane-space.tistory.com/200
 - 챗봇 생성

In [ ]:
#Initial Setting (font, display)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

import matplotlib
get_ipython().run_line_magic('config', "InlineBackend.figure_format='retina'") #화질 보정

In [ ]:
!pip3 install konlpy

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds #SVD
from string import punctuation
from keras import models, layers, optimizers, metrics, preprocessing 
from bs4 import BeautifulSoup

#NLP Preprocessing package
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
print('Ready')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Ready


#### 1) 데이터 수집

In [ ]:
#챗봇의 질의응답 데이터 불러오기
chatbot = pd.read_csv("ChatData.csv")
question = chatbot["Q"]
answer = chatbot["A"]

question = question.to_numpy()
answer = answer.to_numpy()

#데이터 양이 너무 많아 RAM 다운 에러 발생 (데이터 양 줄이기)
question = question[:5000]
answer = answer[:5000]

print(len(question))
print(len(answer))

5000
5000


In [ ]:
for i in range(5): #5개 임의 출력
  print("질문: " + question[i])
  print("답변: " + answer[i])
  print("  ")

질문: 12시 땡!
답변: 하루가 또 가네요.
  
질문: 1지망 학교 떨어졌어
답변: 위로해 드립니다.
  
질문: 3박4일 놀러가고 싶다
답변: 여행은 언제나 좋죠.
  
질문: 3박4일 정도 놀러가고 싶다
답변: 여행은 언제나 좋죠.
  
질문: PPL 심하네
답변: 눈살이 찌푸려지죠.
  


#### 2) Tagging 지정
- Padding
- Start
- End
- OOV

In [ ]:
#태그할 단어
PADDING = "<PADDING>"  #패딩 
START = "<START>"    #시작 
END = "<END>"      #끝 
OOV = "<OOV>"      #out of vocabulary 

PADDING_INDEX = 0
START_INDEX = 1 
END_INDEX = 2 
OOV_INDEX = 3 

ENCODER_INPUT = 0
DECODER_INPUT = 1
DECODER_OUTPUT = 2

#### 3) 데이터 전처리
- HTML 태그 제거
- 정규표현식 이용해서 특수문자 제거
- Okt 형태소 분석기 이용해서 형태소 추출

In [ ]:
from konlpy.tag import Okt 

In [ ]:
def data_preprocess(sentences):
    #텍스트 정제 (HTML 태그 제거)
    for i, sentence in enumerate(sentences):
        sentence = BeautifulSoup(sentence, 'html.parser').text 
        sentences[i] = sentence

    #텍스트 정제 (특수기호 제거)
    for i, sentence in enumerate(sentences):
        sentence = re.sub(r'[^ ㄱ-ㅣ가-힣]', '', sentence) #특수기호 제거, 정규 표현식
        sentences[i] = sentence

    #텍스트 정제 (형태소 추출)
    for i, sentence in enumerate(sentences):
        okt = Okt()
        clean_words = []
        for word in okt.morphs(sentence): 
            clean_words.append(word)
        sentence = ' '.join(clean_words)
        sentences[i] = sentence

    return sentences

In [ ]:
question = data_preprocess(question)
answer = data_preprocess(answer)
print(question[:3])
print(answer[:3])

['시 땡' '지망 학교 떨어졌어' '박일 놀러 가고 싶다']
['하루 가 또 가네요' '위로 해 드립니다' '여행 은 언제나 좋죠']


In [ ]:
#질문+대답 1개로 합치기
sentences = []
sentences.extend(question)
sentences.extend(answer)
print("질문/대답 합친 문장 길이: ", len(sentences))

질문/대답 합친 문장 길이:  10000


#### 4) 단어배열 생성
- word2index: 단어 index화
- index2word: index 단어화
- 딕셔너리 구조 사용

In [ ]:
#단어배열 생성
words = []

for sentence in sentences: #문장 단어 토큰화
  for word in sentence.split():
    words.append(word)


words = [ word for word in words if len(word) > 0] #길이 0인 단어 삭제
words = list(set(words)) #중복단어 삭제

#제일 앞에 태그 단어 삽입
words = [PADDING, START, END, OOV] + words
print(words[:10])
vocab_size = len(words)

['<PADDING>', '<START>', '<END>', '<OOV>', '치즈', '뭘', '초미세먼지', '태어나면', '단거', '모르게']


In [ ]:
#word2index, index2word 생성
word2index = {word:index for index, word in enumerate(words)}
index2word = {index:word for index, word in enumerate(words)}

#### 5) 문장 인코딩
: 주어진 문장에 대해서 인덱스로 변환
- `sentences`: 입력 문장
- `voc`: corpus, 단어사전
- `mytype`: ENCODER_INPUT/DECODER_INPUT/DECODER_TARGET

In [ ]:
maxSequences = 30

def convert_text_to_index(sentences, voc, mytype):
  sentencesIndex = [] #sentenceIndex 모을 빈 리스트

  for sentence in sentences:
    sentenceIndex = [] #인덱스 넣을 빈 리스트
    
    #Decoder 입력일 경우 맨 앞에 START 태그 추가
    if mytype == DECODER_INPUT:
      sentenceIndex.append(voc[START])
  
    #문장 내 단어들 띄어쓰기로 분리
    for word in sentence.split():
      if voc.get(word) is not None: #단어에 해당하는 인덱스 있는 경우
        sentenceIndex.append(voc[word]) #단어에 해당하는 인덱스 추가
      else: #사전에 없는 경우, OOV에 추가
        sentenceIndex.append(voc[OOV])

    #최대 길이 검사
    if mytype == DECODER_OUTPUT:
      #Decoder 출력은 맨 마지막 END 태그 추가
      if len(sentenceIndex) >= maxSequences:
        sentenceIndex = sentenceIndex[:maxSequences-1] + [voc[END]]
      else:
        sentenceIndex += [voc[END]]

    else:
      if len(sentenceIndex) > maxSequences:
        sentenceIndex = sentenceIndex[:maxSequences] 

    #최대 길이 미치지 못하는 문장의 경우, 빈 공간을 0으로 채우기(pad_sequence)
    sentenceIndex += [word2index[PADDING]] * (maxSequences - len(sentenceIndex))
    sentencesIndex.append(sentenceIndex)

  return np.asarray(sentencesIndex)

In [ ]:
#인코더 입력, 디코더 입력, 디코더 출력 > 인덱스 변환

#1. Encoder 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word2index, ENCODER_INPUT)
print(x_encoder[0]) #12시 땡

#2. Decoder 입력 인덱스 변환(START ~)
x_decoder = convert_text_to_index(answer, word2index, DECODER_INPUT)
print(x_decoder[0]) #START 하루 가 또 가네요

#3. Decoder 목표 인덱스 변환(~END)
y_decoder = convert_text_to_index(answer, word2index, DECODER_OUTPUT)
print(y_decoder[0]) #하루 가 또 가네요 END

[1153 2242    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[   1 1210 3893 1114 5430    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[1210 3893 1114 5430    2    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


#### 6) 원핫 인코딩
: Decoder 목표를 원핫 인코딩으로 변환

In [ ]:
import tensorflow as tf

y_decoder = tf.keras.utils.to_categorical(y_decoder, vocab_size)

#### 7) LSTM 훈련 모델 생성

In [ ]:
#LSTM 모델 정의 (Encoder-Decoder 같이 있음)

encoder_input = tf.keras.layers.Input(shape=(None,)) #입력 문장의 인덱스 시퀀스를 입력으로 받음
decoder_input = tf.keras.layers.Input(shape=(None,)) #목표 문장의 인덱스 시퀀스를 입력으로 받음

#Embedding 계층 (encoder_input)
net = tf.keras.layers.Embedding(input_dim=vocab_size, 
                                output_dim=100)(encoder_input)

#state_h =hidden state, state_c =cell state
net, state_h, state_c = tf.keras.layers.LSTM(units=128, 
                                             return_sequences=True, 
                                             return_state=True,
                                             dropout=0.1,
                                             recurrent_dropout=0.5)(net)

#Embedding 계층 (decoder_input)
net = tf.keras.layers.Embedding(input_dim=vocab_size, 
                                output_dim=100)(decoder_input)
net, state_h, state_c = tf.keras.layers.LSTM(units=128, 
                                             return_sequences=True, 
                                             return_state=True,
                                             dropout=0.1,
                                             recurrent_dropout=0.5)(net, initial_state=[state_h, state_c]) #initial_state를 인코더의 상태로 초기화

net = tf.keras.layers.Dense(units=vocab_size, 
                            activation='softmax')(net) #단어의 개수만큼 노드의 개수를 설정해 원핫 형식으로 각 단어 인덱스를 출력

model = tf.keras.models.Model([encoder_input, decoder_input], net)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    620000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    620000      input_2[0][0]                    
______________________________________________________________________________________________

#### 8) 모델 학습 및 검증

In [ ]:
#인코더
encoder_input = model.input[0] #input_1
net = model.layers[2](encoder_input) #embedding
net, state_h, state_c = model.layers[4](net) #lstm
encoder_model = tf.keras.models.Model(encoder_input, 
                                      [state_h, state_c])

encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         620000    
_________________________________________________________________
lstm (LSTM)                  [(None, None, 128), (None 117248    
Total params: 737,248
Trainable params: 737,248
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#디코더
decoder_input = tf.keras.layers.Input(shape=(None,))
state_h_input = tf.keras.layers.Input(shape=(128,))
state_c_input = tf.keras.layers.Input(shape=(128,))   

net = model.layers[-4](decoder_input) #input_3, embedding_1
net, state_h, state_c = model.layers[-2](net, initial_state=[state_h_input, state_c_input]) #input_4, input_5
net = model.layers[-1](net) #lstm_1
decoder_model = tf.keras.models.Model([decoder_input, state_h_input, state_c_input], 
                                      [net, state_h, state_c])

decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    620000      input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 128)]        0                                            
____________________________________________________________________________________________

In [ ]:
#인덱스를 문장으로 변환
def convert_index_to_text(index, voc):
    sentence = ""
    for i in index: #index에 있으면 
        if i == END_INDEX: #종료 인덱스
            break;
        if voc.get(i) is not None: #계속해서 문장으로 변환
            sentence += voc[i]
        else: #index에 없으면 OOV에 추가
            sentence.extend(voc[OOV_INDEX])
        sentence +=" "    
    return sentence  

In [ ]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

for epoch in range(5):
  print("total epoch:", epoch+1)
  history = model.fit([x_encoder, x_decoder], y_decoder,
                      epochs=30,
                      batch_size=64,
                      verbose=0)
  print("accuracy :", history.history['accuracy'])
  print("loss :", history.history['loss'])
    
  #문장 예측 (3박 4일 놀러가고 싶다)
  inputEncoder = x_encoder[2].reshape(1, x_encoder[2].shape[0]) #(30,) > (1,30)
  inputDecoder = x_decoder[2].reshape(1, x_decoder[2].shape[0]) #(30,) > (1,30)
  
  results = model.predict([inputEncoder, inputDecoder])
    
  #결과값에 대해서 가장 큰 값의 위치를 구함
  index = np.argmax(results[0], 1)
  #인덱스 > 문장으로 변환
  sentence = convert_index_to_text(index, index2word)
  print(sentence)
  print()

total epoch: 1
accuracy : [0.8366600275039673, 0.8393599987030029, 0.8417133092880249, 0.8437733054161072, 0.8458933234214783, 0.848026692867279, 0.8499533534049988, 0.851859986782074, 0.8542066812515259, 0.8562866449356079, 0.858513355255127, 0.8608533143997192, 0.8626400232315063, 0.8644333481788635, 0.8661400079727173, 0.8681533336639404, 0.8697800040245056, 0.8716999888420105, 0.8735399842262268, 0.8752933144569397, 0.87691330909729, 0.8785333037376404, 0.880079984664917, 0.8819266557693481, 0.8838133215904236, 0.8851733207702637, 0.8867800235748291, 0.888426661491394, 0.8895666599273682, 0.8917333483695984]
loss : [1.1449283361434937, 1.0701204538345337, 1.0320361852645874, 1.0041048526763916, 0.9797017574310303, 0.9571412801742554, 0.9359521269798279, 0.9156511425971985, 0.8947233557701111, 0.8733197450637817, 0.8512698411941528, 0.8302785158157349, 0.8106818199157715, 0.7915635108947754, 0.7738949656486511, 0.7566031217575073, 0.7402693629264832, 0.7238367199897766, 0.7093065977

#### 9) 문장 예측 / 예측 답변 생성

In [ ]:
def make_predict_input(sentence):    
    sentences=[]
    sentences.append(sentence)
    sentences = data_preprocess(sentences) #전처리
    inputSeq = convert_text_to_index(sentences, #인덱스화
                                word2index,
                                ENCODER_INPUT)
    return inputSeq
    
make_predict_input("3박4일 놀러가고 싶다") #인덱스된 문장

array([[4514, 4994, 3277, 6157,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [ ]:
def generate_text(inputSeq):
  #입력을 인코더에 넣고, 마지막 상태 구함
  states = encoder_model.predict(inputSeq)
        
  #목표 시퀀스 초기화
  targetSeq = np.zeros((1,1))
  #<START> 시그널 추가
  targetSeq[0,0] = START_INDEX
  #인덱스 초기화
  indexs=[]
        
  #입력되는 텍스트에 대해 디코더 반복
  while 1:
    decoderOuputs, stateH, stateC = decoder_model.predict([targetSeq]+states)
    #결과를 원핫인코딩 형식으로 변환
    index = np.argmax(decoderOuputs[0,0,:])
    indexs.append(index)
    #종료 체크
    if index == END_INDEX or len(indexs) >= maxSequences:
      break
                
    #targetSeq를 이전 출력으로 설정
    targetSeq = np.zeros((1,1))
    targetSeq[0,0] = index    #START_INDEX 
            
    #디코더의 이전 상태를 다음 디코더 예측에 사용
    states=[stateH, stateC]
            
    #인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index2word)  
    return sentence

In [ ]:
inputSeq = make_predict_input("안녕")
sentence = generate_text(inputSeq)
sentence

'안녕하세요 '

#### 한계점
1. question, answer을 각각 5,000개만 사용함 (전체 데이터셋 크기:12,000이지만 다 돌리려고 하니까 계속 RAM이 폭파하는 문제 발생)
2. epoch을 30번씩 5번밖에 돌리지 못함, default가 100번씩 10번이었음<br>
*챗봇 성능이 아직 예상했던 것만큼 좋지 않음*